## Beyond exceptions: `Result`

We have seen that the exceptions thrown by a program can alter the workflow of the program. In the case in which the exceptions are not caught, the program simply terminates, with consequent damage to the user.

We also saw that we have the possibility of issuing our own exceptions, without using the operating system's own.

In the previous example, we had a `divide` function that raised certain types of exceptions according to cases that are errors (either because it involves performing a prohibited operation), or cases in which the use case is not allowed by the system (like dividing by one).




In [1]:
exception DivideByOne of string 

let divide x y =
    match y with 
    | 0.0 -> invalidOp "Trying to divide by zero!!" 
    | 1.0 -> raise (DivideByOne "Can't believe you are trying to divide by one")
    | _ -> x/y 


On the other hand, we had the `annoyedDivide` function, which is the one that calls `divide` and is in charge of capturing possible errors through exceptions:

In [2]:
let annoyedDivide x y =
    try 
        divide x y 
    with 
    | DivideByOne s -> 
        printfn "%A" (s)
        -1.0
    | :? System.InvalidOperationException as ex -> 
        printfn "Message: %A" (ex.Message)
        -1.0          

        

What would happen if, in addition to dividing two numbers, we had to calculate the square root of the result? We would have to use the `sqrt` function

In [3]:
printfn $"Sqrt 4: {sqrt 4.0}"
printfn $"Sqrt 4: {sqrt -4.0}"

Sqrt 4: 2
Sqrt 4: NaN


But we have to validate that the input of the function is not negative:

In [5]:
exception CannotSqrtOfNegative of string 

let squareRoot (x: float) = 
    if x < 0 then 
        raise (CannotSqrtOfNegative "Cannot compute sqrt of a negative number")
    else 
        sqrt x         

In [6]:
printfn "%A" (squareRoot 4.0)
printfn "%A" (squareRoot -4.0)

2.0


Error: FSI_0009+CannotSqrtOfNegative: CannotSqrtOfNegative "Cannot compute sqrt of a negative number"
   at FSI_0009.squareRoot(Double x)
   at <StartupCode$FSI_0010>.$FSI_0010.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In [7]:
let annoyedSqrt x =
    try 
        squareRoot x 
    with 
    | CannotSqrtOfNegative s -> 
        printfn "Message: %A" s
        -1 


Once again we run into the problem of what to return when the exception is caught.

In [8]:
let x = 5.0
let y = 3.0 

sqrt (x/y)

1.2909944487358056

In [9]:
let x = 5
let y = 0 

sqrt  (annoyedDivide x y)

Message: "Trying to divide by zero!!"


NaN

In [10]:
let x = -5
let y = 3 

annoyedSqrt  (annoyedDivide x y)

Message: "Cannot compute sqrt of a negative number"


-1

In [11]:
let x = -5
let y = 3 

annoyedDivide x y
|> annoyedSqrt

Message: "Cannot compute sqrt of a negative number"


-1

### The `Result` type

To overcome the drawbacks of this approach to error handling, F# provides a data type for this purpose. Note that both `divide` and `squareRoot` have two possible outputs, depending on whether the calculation can be performed or if there is an error. In no case can one be in both situations at the same time, that is, they are _disjoint_ situations. That's what discriminated unions are for!

The language defines a particular union type, called `Result`:

```fsharp
type Result<'T,'TError> =
    | Ok of 'T
    | Error of 'TError
```

`Result is a generic type that can be in one of two possible states
- `Ok 'T` when the operation could be performed
- `Error 'TError` when there is an error.

In [13]:
let divideR x y =
    match y with 
    | 0.0 -> Error "Trying to divide by zero!!" 
    | 1.0 -> Error "Can't believe you are trying to divide by one"
    | _ -> Ok (x/y) 

printfn "%A" (divideR 4 2)
printfn "%A" (divideR 4 1)
printfn "%A" (divideR 4 0)

Ok 2.0
Error "Can't believe you are trying to divide by one"
Error "Trying to divide by zero!!"


In [14]:
let squareRootR (x: float) = 
    if x < 0 then 
        "Cannot compute sqrt of a negative number" |> Error 
    else 
        sqrt x |> Ok       

In [15]:
printfn "%A" (squareRootR 4.0)
printfn "%A" (squareRootR -4.0)

Ok 2.0
Error "Cannot compute sqrt of a negative number"


#### Railway programming

Obviously, the use of the `Result` type clarifies the code significantly. But now it happens that:

```fsharp
divideR:
   x: float ->
   y: float
   -> Result<float,string>
```

while

```fsharp 
squareRootR:
   x: float
   -> Result<float,string>
```

So it is not possible for me to compose both functions, since the output of `divideR` is of type `Result`, while the input of `squareRootR` is a `float`.




In [16]:
let x = 4
let y = 6 

divideR x y 
|> Result.bind squareRootR

ResultValue,0.816496580927726
ErrorValue,<null>


In [17]:
let x = 4
let y = 0 

divideR x y 
|> Result.bind squareRootR

ResultValue,0
ErrorValue,Trying to divide by zero!!


In [18]:
let x = -4
let y = 4 

divideR x y 
|> Result.bind squareRootR

ResultValue,0
ErrorValue,Cannot compute sqrt of a negative number


The `bind` method of `Result` takes care of pairing the inputs and outputs in order to connect both functions:

```fsharp 
bind:
   binder: ('T -> Result<'U,'TError>) ->
   result: Result<'T,'TError>
        -> Result<'U,'TError>
bind f inp evaluates to match inp with Error e -> Error e | Ok x -> f x
```

Now suppose that we also want to add a value to the result of $\sqrt{x/y}$. Fortunately, the addition is a less restrictive operation than the previous ones, so it is not necessary so much ceremony:

In [19]:
let add10 y =
    y + 10.0 

But now we have another problem, because our entire operation outputs `Result<float,string>`, which is not the input of `add10`....

For that there is the `map` method of `Result`:


In [20]:
let x = 8
let y = 2 

divideR x y 
|> Result.bind squareRootR
|> Result.map add10 

ResultValue,12
ErrorValue,<null>


In [21]:
let x = 8
let y = -2 

divideR x y 
|> Result.bind squareRootR
|> Result.map add10 

ResultValue,0
ErrorValue,Cannot compute sqrt of a negative number


In [22]:
let x = 8
let y = 0 

divideR x y 
|> Result.bind squareRootR
|> Result.map add10 

ResultValue,0
ErrorValue,Trying to divide by zero!!


In [23]:
let x = 8
let y = 0 

let res = 
    divideR x y 
    |> Result.bind squareRootR
    |> Result.map add10 

res
|> Result.mapError (fun e -> $"Hubo un error: {e}")    

ResultValue,0
ErrorValue,Hubo un error: Trying to divide by zero!!
